# "Community Detection"
## on sample social graph
### via "NetworkX" and "community-louvain" (aka python-louvain) library

In [ ]:
!pip install networkx python-louvain matplotlib

# CLASSIC Approach: Louvain Detector

In [ ]:
import networkx as nx
import community as community_louvain

def detect_communities_louvain(G):
    """
    Run Louvain algorithm on graph G and return partition.
    G: networkx.Graph
    Returns: dict mapping node → community_id
    """
    partition = community_louvain.best_partition(G)
    return partition

# Plot

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import matplotlib.cm as cm

def draw_partitioned_graph(G, partition, title="Louvain Community Detection"):
    """
    Visualize the graph with node colors based on community partition.
    """
    pos = nx.spring_layout(G, seed=42)
    cmap = cm.get_cmap('Set3', max(partition.values()) + 1)

    nx.draw_networkx_nodes(G, pos, node_size=300,
                           node_color=list(partition.values()),
                           cmap=cmap)
    nx.draw_networkx_edges(G, pos, alpha=0.4)
    nx.draw_networkx_labels(G, pos, font_size=10)
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

# Main Louvain

In [ ]:
import networkx as nx
from community_detection.louvain_detector import detect_communities_louvain
from utils.plot import draw_partitioned_graph

# Generate test graph (Karate Club)
G = nx.karate_club_graph()

# Run Louvain algorithm
partition = detect_communities_louvain(G)

# Print community results
for node, comm in partition.items():
    print(f"Node {node}: Community {comm}")

# Draw the result
draw_partitioned_graph(G, partition)

# HYBRID Approach
# Apply Node2Vec + Clustering (instead of Louvain) to the Telegram user interaction graph.

🚀 What This Gives Us:

	•	A flexible pipeline to detect communities from our Telegram/X data
	•	A lightweight and scalable alternative to Louvain
	•	A clear visual map of potential echo chambers
	•	Ready for sentiment analysis inside each community (next step in echo chamber detection)

# 🔹 1. Build the Telegram interaction graph

In [ ]:
messages = [
    {"sender": "user1", "reply_to": "user3"},
    {"sender": "user2", "reply_to": "user1"},
    {"sender": "user3", "reply_to": "user2"},
    # ...
]

## 📄 Build Graph from Message Data

In [ ]:
import networkx as nx

def build_user_graph(messages):
    G = nx.DiGraph()  # or use nx.Graph() if interactions are symmetric
    for msg in messages:
        sender = msg.get("sender")
        replied = msg.get("reply_to")
        if sender and replied:
            if G.has_edge(sender, replied):
                G[sender][replied]['weight'] += 1
            else:
                G.add_edge(sender, replied, weight=1)
    return G

# 🔹 2. Generate Node2Vec Embeddings

In [ ]:
from node2vec import Node2Vec

def get_node_embeddings(G, dimensions=64):
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=20, num_walks=100, workers=2)
    model = node2vec.fit(window=10, min_count=1)
    embeddings = [model.wv[str(node)] for node in G.nodes()]
    return embeddings, list(G.nodes())

# 🔹 3. Apply Clustering (KMeans)

In [ ]:
from sklearn.cluster import KMeans

def run_kmeans(embeddings, n_clusters=4):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(embeddings)
    return labels

# 🔹 4. Visualize Results

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def visualize_embeddings(embeddings, labels, nodes):
    reduced = PCA(n_components=2).fit_transform(embeddings)
    plt.figure(figsize=(8,6))
    for label in set(labels):
        idx = [i for i, l in enumerate(labels) if l == label]
        x = [reduced[i][0] for i in idx]
        y = [reduced[i][1] for i in idx]
        plt.scatter(x, y, label=f"Community {label}", s=100)
    for i, node in enumerate(nodes):
        plt.text(reduced[i][0], reduced[i][1], str(node), fontsize=8)
    plt.title("Detected Communities (Node2Vec + KMeans)")
    plt.legend()
    plt.tight_layout()
    plt.show()

# 🔚 5. Glue It All Together

In [ ]:
# Sample message data (replace with real extracted messages)
messages = [
    {"sender": "alice", "reply_to": "bob"},
    {"sender": "carol", "reply_to": "alice"},
    {"sender": "bob", "reply_to": "carol"},
    # Add your real data here
]

G = build_user_graph(messages)
embeddings, nodes = get_node_embeddings(G)
labels = run_kmeans(embeddings, n_clusters=3)
visualize_embeddings(embeddings, labels, nodes)